In [23]:
! pip install replicate requests aiohttp python-dotenv Pillow pyfs

^C


In [ ]:
import asyncio
from collections import defaultdict
import json
import aiohttp
import requests
import os
from dotenv import load_dotenv

load_dotenv()

last_fm_api_base_url = "http://ws.audioscrobbler.com/2.0/"


def get_top_200_last_fm_artists():
    top_200_last_fm_artists = []
    for page_number in range(1, 5):
        fifty_top_artists = requests.get(last_fm_api_base_url + "?method=chart.gettopartists&api_key=" + os.environ["LAST_FM_API_KEY"] + "&format=json&page=" + str(page_number)).json()
        for artist in fifty_top_artists["artists"]["artist"]:
            top_200_last_fm_artists.append(artist["name"])
    return top_200_last_fm_artists



def get_top_three_albums_for_all_artists(artists):
    top_three_albums = defaultdict(list)
    for artist in artists:
        top_three_albums[artist] = [album["name"] for album in requests.get(last_fm_api_base_url + "?method=artist.gettopalbums&artist=" + artist + "&api_key=" + os.environ["LAST_FM_API_KEY"] + "&format=json&limit=3").json()["topalbums"]["album"]]
    return top_three_albums


def generate_urls_to_get_top_three_albums(top_three_albums_for_artists):
    urls = []
    for artist, albums in top_three_albums_for_artists.items():
        for album in albums:
            urls.append(last_fm_api_base_url + "?method=album.getinfo&api_key=" + os.environ["LAST_FM_API_KEY"] + "&artist=" + artist + "&album=" + album + "&format=json")
    return urls


urls = generate_urls_to_get_top_three_albums(get_top_three_albums_for_all_artists(get_top_200_last_fm_artists()))

json.dump(urls, open("urls.json", "w+"), indent = 4, ensure_ascii=False)

In [ ]:
async def get_album_with_info(session, url):
    async with session.get(url) as response:
        return await response.json()


async def get_albums_with_info(urls):
    async with aiohttp.ClientSession() as session:
        albums_with_info = [get_album_with_info(session, url) for url in urls]
        return await asyncio.gather(*albums_with_info)




In [36]:
import json
import replicate
from PIL import Image, ImageChops
import fs
from io import BytesIO
import os
import requests

os.environ["REPLICATE_API_TOKEN"]
input = {
    "image": 'https://th.bing.com/th/id/OIP.QT_ze_xwLmxbepdySxndMQHaHa?rs=1&pid=ImgDetMain',
    "mask_limit": 5,
    "mask_2_mask": False, 
    "crop_n_layers": 0,
    "points_per_side": 64,
    "stability_score_thresh": 0.5,
}

output = replicate.run(
    "lucataco/segment-anything-2:be7cbde9fdf0eecdc8b20ffec9dd0d1cfeace0832d4d0b58a071d993182e1be0",
    input=input
)

input_response = requests.get(input.get("image"))
input_image = Image.open(BytesIO(input_response.content)).convert("RGBA")

for i, mask_url in enumerate(output):
    output_response = requests.get(mask_url)
    mask = Image.open(BytesIO(output_response.content)).convert("L")  # Convert to grayscale

    bbox = mask.getbbox()
    if bbox:
        non_transparent_area = mask.crop(bbox)
        print(f"Mask {i} non-transparent area dimensions: {non_transparent_area.size}")
    else:
        print(f"Mask {i} is fully transparent")

    # Create a binary mask
    binary_mask = mask.point(lambda p: p > 128 and 255)

    # Composite the images using the binary mask
    composite_image = Image.composite(input_image, Image.new("RGBA", input_image.size), binary_mask)

    # Save or show the composite image
    composite_image.show()
    composite_image.save(f"composite_image_{i}.png")

print('hi', input.get("image"))
print(json.dumps(output))

Mask 0 dimensions: (474, 474)
Mask 1 dimensions: (474, 474)
Mask 2 dimensions: (474, 474)
Mask 3 dimensions: (474, 474)
Mask 4 dimensions: (474, 474)
hi https://th.bing.com/th/id/OIP.QT_ze_xwLmxbepdySxndMQHaHa?rs=1&pid=ImgDetMain
["https://replicate.delivery/pbxt/2ie5Y5EPJTTofU2TKhM3va6CHRgiA96W3pOf5fczJKJh6i8MB/mask_0.png", "https://replicate.delivery/pbxt/RzRredSa2cQGHKeokkmH40u7nX9seRECPUzoO91O9EzSdReMB/mask_1.png", "https://replicate.delivery/pbxt/e6T8xv0rXnzpMiuot5mvjEUfw0tAlDIrS4qWA0cE5VeSdReMB/mask_2.png", "https://replicate.delivery/pbxt/m8P19MmSOa7uPdqiGPeAfxMzfXGP5uyDX3ZivoBOHQUTdReMB/mask_3.png", "https://replicate.delivery/pbxt/JBVBY8UiefnWzEMByVoIqu9zgJqyiOELNb5fcJjlUnoTdReMB/mask_4.png"]
